In [1]:
from astropy.time import Time, TimeDelta
import h5py
import numpy as np
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec
from pprint import pprint
# plt.style.use('dark_background')

import sorts
import pyorb
import pyant

HERE = Path('.').parent.resolve()
OUTPUT = HERE / 'output' / 'russian_asat'
print(f'Using {OUTPUT} as output')

Using /home/danielk/git/antisat_analysis/projects/output/russian_asat as output


In [2]:
with open(OUTPUT / 'paths.pickle', 'rb') as fh:
    paths = pickle.load(fh)

In [3]:
category_files = {}
dates = {}
for radar in paths['data_paths']:
    category_files[radar] = []
    dates[radar] = []
    for ind in range(len(paths['data_paths'][radar])):
        date = paths['data_paths'][radar][ind].stem.replace('.', '-')
        dates[radar].append(date)
        out_categories = OUTPUT / f'{date}_{radar}_categories.npy'
        category_files[radar].append(out_categories)
pprint(category_files)

{'esr': [PosixPath('/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021-11-19_esr_categories.npy'),
         PosixPath('/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021-11-23_esr_categories.npy'),
         PosixPath('/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021-11-25_esr_categories.npy'),
         PosixPath('/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021-11-29_esr_categories.npy')],
 'uhf': [PosixPath('/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021-11-23_uhf_categories.npy'),
         PosixPath('/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021-11-25_uhf_categories.npy'),
         PosixPath('/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021-11-29_uhf_categories.npy')]}


In [5]:
save_paths = {}
for radar in dates:
    save_paths[radar] = []
    for date in dates[radar]:
        save_path = OUTPUT / f'{date}_{radar}_kosmos_stat'
        save_path.mkdir(exist_ok=True)
        save_paths[radar].append(save_path)

In [6]:
radar_title = {
    'esr': [
        'EISCAT Svalbard Radar 32m',
        'EISCAT Svalbard Radar 32m',
        'EISCAT Svalbard Radar 42m',
        'EISCAT Svalbard Radar 42m',
    ],
    'uhf': ['EISCAT UHF']*3,
}

In [12]:
def load_data(file):
    with h5py.File(file, "r") as h:
        v = h["v"][()]
        t = h["t"][()]
        r = h["r"][()]    
        snr = h["snr"][()]
        dur = h["dur"][()]
        diam = h["diams"][()]

    return t, r, v, snr, dur, diam

def plot_statistics(data, select, radar, ind, size=(10, 8)):

    t, r, v, snr, dur, diam = data

    bins = int(np.round(np.sqrt(len(r))))

    fig, axes = plt.subplots(2, 2, figsize=size, sharey='all')
    
    axes[0, 0].hist(r[select], bins=bins)
    axes[0, 0].set_xlabel("Range [km]")
    axes[0, 0].set_ylabel("Detections")

    axes[0, 1].hist(v[select], bins=bins)
    axes[0, 1].set_xlabel("Range-rate [km]")
    axes[0, 1].set_ylabel("Detections")

    axes[1, 0].hist(10*np.log10(snr[select]), bins=bins)
    axes[1, 0].set_xlabel("SNR [dB]")
    axes[1, 0].set_ylabel("Detections")

    _diam = diam[select]
    sel = np.logical_or(np.isinf(_diam), np.isnan(_diam))
    sel = np.logical_not(sel)
    _diam = _diam[sel]

    axes[1, 1].hist(np.log10(_diam), bins=bins)
    axes[1, 1].set_xlabel("Minimum diameter [log10(cm)]")
    axes[1, 1].set_ylabel("Detections")

    fig.suptitle(f'KOSMOS-1408 fragments {radar_title[radar][ind]} {dates[radar][ind]}')
    fig.savefig(save_path / f'{date}_{radar}_stat.png')
    plt.close(fig)

In [14]:
class Categories:
    uncorrelated_other = 0
    correlated_other = 1
    uncorrelated_kosmos = 2
    correlated_kosmos = 3

def kosmos_select(select):
    return np.logical_or(
        select == Categories.uncorrelated_kosmos,
        select == Categories.correlated_kosmos,
    )

In [15]:
radar = 'uhf'
ind = 1
select = np.load(category_files[radar][ind])
data_file = paths['data_paths'][radar][ind]
plot_statistics(load_data(data_file), kosmos_select(select), radar, ind)